# Eksempler på bruk av Klass fra Python 
Klass - Klassifikasjoner og kodelister. 
Se også Klass API dokumentasjon: [Klass API guide](https://data.ssb.no/api/klass/v1/api-guide.html)

### 1. Et enkelt eksempel: Dagens kommuner

Vi importerer Python Pandas, som kan angis som pd

In [1]:
# Pandas gjør dataanalyse i Python enklere
import pandas as pd

Eksempelet henter gyldige koder fra Standard for kommuneinndeling.

Adressen i Klass er https://www.ssb.no/klass/klassifikasjoner/131/

131 = ID til klassifikasjonen

Vi henter gyldige koder som JSON via Klass API med [codesAt](https://data.ssb.no/api/klass/v1/api-guide.html#_codesat):

In [2]:
URL1 = 'http://data.ssb.no/api/klass/v1/classifications/131/codesAt.json?date=2021-01-19'

Siden dette er json kan vi bruke Pandas read_json(). Den kan ta nettadresser direkte.

In [3]:
dataframe1 = pd.read_json(URL1)
dataframe1

,codes
0,"{'code': '0301', 'parentCode': None, 'level': ..."
1,"{'code': '1101', 'parentCode': None, 'level': ..."
2,"{'code': '1103', 'parentCode': None, 'level': ..."
3,"{'code': '1106', 'parentCode': None, 'level': ..."
4,"{'code': '1108', 'parentCode': None, 'level': ..."
...,...
352,"{'code': '5441', 'parentCode': None, 'level': ..."
353,"{'code': '5442', 'parentCode': None, 'level': ..."
354,"{'code': '5443', 'parentCode': None, 'level': ..."
355,"{'code': '5444', 'parentCode': None, 'level': ..."


Alt ligger i en kolonne 'codes'. Vi må i tillegg bruke Pandas [json_normalize](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html). 

Vi gir dataframe1\['codes'\] som input

In [4]:
kommuner2020 = pd.json_normalize(dataframe1['codes'])
kommuner2020

,code,parentCode,level,name,shortName,presentationName
0,0301,None,1,Oslo,,
1,1101,None,1,Eigersund,,
2,1103,None,1,Stavanger,,
3,1106,None,1,Haugesund,,
4,1108,None,1,Sandnes,,
...,...,...,...,...,...,...
352,5441,None,1,Deatnu - Tana,,
353,5442,None,1,Unjárga - Nesseby,,
354,5443,None,1,Båtsfjord,,
355,5444,None,1,Sør-Varanger,,


### 2. Henter Standard for kommuneinndeling med endringer fra og med  1.1.2017 
 
Url til Klass API. Nå bruker vi [codes](https://data.ssb.no/api/klass/v1/api-guide.html#_codes) med fra-og-med dato. Id er fortsatt 131 og vi angir .json slik: 

In [5]:
URL2 = 'http://data.ssb.no/api/klass/v1/classifications/131/codes.json?from=2017-01-01'

to= parameteren for tid er ikke nødvendig i Klass API. Den kan med fordel utelukkes om en vil ha med endringer. 

Leser URL2 som json til dataframe2

In [6]:
dataframe2 = pd.read_json(URL2)
dataframe2.head()

,codes
0,"{'code': '0101', 'parentCode': None, 'level': ..."
1,"{'code': '0104', 'parentCode': None, 'level': ..."
2,"{'code': '0105', 'parentCode': None, 'level': ..."
3,"{'code': '0106', 'parentCode': None, 'level': ..."
4,"{'code': '0111', 'parentCode': None, 'level': ..."


Vi bruker Pandas json_normalize

In [7]:
kommuner_fra2017 = pd.json_normalize(dataframe2['codes'])
kommuner_fra2017

,code,parentCode,level,name,shortName,presentationName,validFromInRequestedRange,validToInRequestedRange
0,0101,None,1,Halden,,,2017-01-01,2020-01-01
1,0104,None,1,Moss,,,2017-01-01,2020-01-01
2,0105,None,1,Sarpsborg,,,2017-01-01,2020-01-01
3,0106,None,1,Fredrikstad,,,2017-01-01,2020-01-01
4,0111,None,1,Hvaler,,,2017-01-01,2020-01-01
...,...,...,...,...,...,...,...,...
725,5441,None,1,Deatnu - Tana,,,2020-01-01,None
726,5442,None,1,Unjárga - Nesseby,,,2020-01-01,None
727,5443,None,1,Båtsfjord,,,2020-01-01,None
728,5444,None,1,Sør-Varanger,,,2020-01-01,None


Vi bruker drop for å fjerne unødvendige kolonner.

In [9]:
kommuner_fra2017 = kommuner_fra2017.drop(columns=['parentCode', 'level', 'shortName', 'presentationName'])
kommuner_fra2017.head()

,code,name,validFromInRequestedRange,validToInRequestedRange
0,0101,Halden,2017-01-01,2020-01-01
1,0104,Moss,2017-01-01,2020-01-01
2,0105,Sarpsborg,2017-01-01,2020-01-01
3,0106,Fredrikstad,2017-01-01,2020-01-01
4,0111,Hvaler,2017-01-01,2020-01-01


Setter code som inneholder kommunenummrene som index

In [10]:
kommuner_fra2017 = kommuner_fra2017.set_index('code')

In [11]:
kommuner_fra2017.tail()

,name,validFromInRequestedRange,validToInRequestedRange
code,,,
5441,Deatnu - Tana,2020-01-01,None
5442,Unjárga - Nesseby,2020-01-01,None
5443,Båtsfjord,2020-01-01,None
5444,Sør-Varanger,2020-01-01,None
9999,Uoppgitt,2017-01-01,None


Finn kommunenummer og gyldighetsperiode for Holmestrand. Se gyldighetsperiodene.

In [12]:
kommuner_fra2017.query('name == "Holmestrand"')

,name,validFromInRequestedRange,validToInRequestedRange
code,,,
0702,Holmestrand,2017-01-01,2018-01-01
0715,Holmestrand,2018-01-01,2020-01-01
3802,Holmestrand,2020-01-01,None


Spørsmål? Kontakt Jan Bruusgaard [jbr@ssb.no](mailto:jbr@ssb.no) eller [statistikkbanken@ssb.no](statistikkbanken@ssb.no)